### Harmonize survey results

#### Seperates the survey data into object counts, uses location and date as survey identifier.


Fetches data from the local source and esyablishes the following variables:

1. dutch_codes
2. swiss_codes
3. dutch_surveys
4. swiss_surveys
5. swiss_beaches


Establishes directory variables for fetching and putting to all subdirectories:

1. data
2. beaches
3. codes
4. geo
5. output

provides a script to update the remote data.

In [1]:
# sys things
import os
import sys
import json

# networks
import requests

# data
import pandas as pd
import numpy as np
import datetime as dt
import scipy
from statsmodels.distributions.empirical_distribution import ECDF
from scipy.stats import kendalltau, pearsonr, spearmanr
import math

# charting:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import ticker
import matplotlib.ticker as mtick
import seaborn as sns
from matplotlib.gridspec import GridSpec
from IPython.display import display, Markdown, Latex, HTML

# mapping
import geopandas as gpd

# home brew utitilties
import resources.utilities.utility_functions as ut

# documenting
from IPython.display import Markdown as md


# returns the p_value for each test
def kendall_pval(x,y):
    return kendalltau(x,y)[1]

def pearsonr_pval(x,y):
    return pearsonr(x,y)[1]

def spearmanr_pval(x,y):
    return spearmanr(x,y)[1]

# convenience functions for tables
def make_table_grids(anax):
    anax.grid(False)
    anax.spines["top"].set_visible(False)
    anax.spines["right"].set_visible(False)
    anax.spines["bottom"].set_visible(False)
    anax.spines["left"].set_visible(False)
    return(anax)

def table_fonts(a_table, size=12):
    a_table.auto_set_font_size(False)
    a_table.set_fontsize(size)

# standard formats already in use for charts, these will gradually
# define the chart style or output format for the app
# you can just apply these as kwargs to different elements...
# table kwargs
table_k = dict(loc="top left", bbox=(0,0,1,1), colWidths=[.5, .5], cellLoc='center')
tablecenter_k = dict(loc="top left", bbox=(0,0,1,1), cellLoc='center')
tabtickp_k = dict(axis='both', which='both', bottom=False, top=False, left=False, right=False, labelleft=False, labelbottom=False)

# chart kwargs
title_k = {'loc':'left', 'pad':12, 'linespacing':1.5, 'fontsize':12}
title_k14 = {'loc':'left', 'pad':12, 'linespacing':1.5, 'fontsize':14}
title_k20 = {'loc':'left', 'pad':10, 'linespacing':1.5, 'fontsize':12, 'color':'dodgerblue'}
title_k17 = {'loc':'left', 'pad':10, 'linespacing':1.5, 'fontsize':12, 'color':'salmon'}
titler_k20 = {'loc':'right', 'pad':10, 'linespacing':1.5, 'fontsize':12, 'color':'dodgerblue'}
titler_k17 = {'loc':'right', 'pad':10, 'linespacing':1.5, 'fontsize':12, 'color':'salmon'}
xlab_k = {'labelpad':10, 'fontsize':12}
ylab_k = {'labelpad':14, 'fontsize':14}
titler_k = {'loc':'right', 'pad':10, 'linespacing':1.5, 'fontsize':12}
label45r = {'rotation':45, 'ha':'right'}
label45c = {'rotation':45, 'ha':'center'}

# use these to format date axis in charts
weeks = mdates.WeekdayLocator(byweekday=1, interval=4)
onedayweek = mdates.DayLocator(bymonthday=1, interval=1)
everytwoweeks = mdates.WeekdayLocator(byweekday=1, interval=4)

months = mdates.MonthLocator(bymonth=[3,6,9,12])
bimonthly = mdates.MonthLocator(bymonth=[1,3,5,7,9,11])
allmonths = mdates.MonthLocator()
wks_fmt = mdates.DateFormatter('%d')
mths_fmt = mdates.DateFormatter('%b')

In [2]:
# get folder extesions
data, beaches, codes, geo, output=ut.make_local_paths()
print("look for resources here\n")
print(data, beaches, codes, geo, output)

look for resources here

resources/surveydata resources/locationdata resources/mlwcodedefs resources/geodata output


In [3]:
project_directory = ut.make_project_folder(output, "harmonized_results")
print("put stuff here:\n")
print(project_directory)

put stuff here:

output/harmonized_results


In [4]:
# survey_data
dutch_surveys = pd.read_csv(data+'/dataset_macrolitter_NL.csv')

# use the aggregated hd data. This accounts for the custom codes used in Switzerland
swiss_surveys = pd.read_csv(data+'/aggregated_hd_surveys.csv')

# location data
swiss_beaches = pd.read_csv(beaches+'/hammerdirt_beaches.csv')

In [5]:
print("Columns from cleaned up dutch data\n")
print(dutch_surveys.columns)

print("\nColumns from cleaned up swiss data\n")
print(swiss_surveys.columns)

Columns from cleaned up dutch data

Index(['Unnamed: 0', 'Gebiedscode', 'Rivier', 'Coordinaten', 'Lat', 'Lon',
       'meting', 'date', 'doy', 'dop',
       ...
       'sanitair_tissues_wc_papier', 'sanitair_toiletverfrissers',
       'sanitair_overig_sanitair', 'medisch_verpakkingen', 'medisch_spuiten',
       'medisch_overig_medisch', 'Unnamed: 121', 'Totaal', 'Items/km',
       'granulaat_korrels'],
      dtype='object', length=125)

Columns from cleaned up swiss data

Index(['loc_date', 'code', 'date', 'pcs_m', 'quantity', 'location',
       'water_name'],
      dtype='object')


In [6]:
print("the first ten rows are admin data\n")
print(dutch_surveys.columns[:10])
print("\nthe next 115 is code data\n")
print(dutch_surveys.columns[10:])
print("\none row is equal to one survey")

the first ten rows are admin data

Index(['Unnamed: 0', 'Gebiedscode', 'Rivier', 'Coordinaten', 'Lat', 'Lon',
       'meting', 'date', 'doy', 'dop'],
      dtype='object')

the next 115 is code data

Index(['plastic_6_packringen', 'plastic_tassen',
       'plastic_kleine_plastic_tasjes',
       'plastic_drankflessen_groterdan_halveliter',
       'plastic_drankflessen_kleinerdan_halveliter',
       'plastic_wikkels_van_drankflessen',
       'plastic_verpakking_van_schoonmaakmiddelen',
       'plastic_voedselverpakkingen_frietbakjes_etc',
       'plastic_cosmeticaverpakkingen',
       'plastic_motorolieverpakking_groterdan50cm',
       ...
       'sanitair_tissues_wc_papier', 'sanitair_toiletverfrissers',
       'sanitair_overig_sanitair', 'medisch_verpakkingen', 'medisch_spuiten',
       'medisch_overig_medisch', 'Unnamed: 121', 'Totaal', 'Items/km',
       'granulaat_korrels'],
      dtype='object', length=115)

one row is equal to one survey


In [7]:
print("This is the number of rows\n")
print(len(dutch_surveys))
print("\nThis is the number of unique 'Gebiedscode' values\n")
print(len(dutch_surveys.Gebiedscode.unique()))

This is the number of rows

391

This is the number of unique 'Gebiedscode' values

212


In [8]:
# key the Gebiedscode to an int
count_gebied = {n:i+1 for i,n in enumerate(dutch_surveys.Gebiedscode.unique())}

# grab only the object count columns and the river
obj_counts = dutch_surveys.columns[10:-4]

In [9]:
dutch_surveys['new date'] = dutch_surveys.date.map(lambda x: F"{x}")

def make_this_a_date(x):
    a_date = F"{x}"
#     print(a_date[:4])
#     print(len(a_date))
    if len(a_date) == 8:
        new_date = F"{a_date[:4]}-{a_date[4:6]}-{a_date[6:]}"
#         print(new_date)
        try:
            a_date = dt.datetime.strptime(new_date, "%Y-%m-%d").date()
        except:
            a_date = 'nothing'
#         print(a_date)
    else:
        a_date = 'nothing'
        
    return a_date

dutch_surveys['this date'] = dutch_surveys['new date'].map(lambda x: make_this_a_date(x))
no_date_dutch_surveys =dutch_surveys[dutch_surveys['this date'] == 'nothing'].copy()
dutch_surveys = dutch_surveys[dutch_surveys['this date'] != 'nothing'].copy()
dutch_surveys['date'] = dutch_surveys['this date']


In [10]:
# these columns are admin data
admin = ['Gebiedscode', 'Rivier', 'Coordinaten', 'Lat', 'Lon', 'meting', 'date', 'doy', 'dop']
date = ['date']

# unique id per row:
loc_date = ['Gebiedscode', 'date']

# these are no longer needed
no_longer_needed = ['Unnamed: 121', 'Totaal', 'Items/km', 'granulaat_korrels']



def make_location_groups(x, keys):
    try:
        name = keys[x]
    except:
        name = x
    return name

# make a column of the keys created for Gebiedscode
dutch_surveys['name'] = dutch_surveys.Gebiedscode.map(lambda x: make_location_groups(x, count_gebied))

# zip that up with the date
dutch_surveys['loc_date'] = list(zip(dutch_surveys.name, dutch_surveys.date))
dutch_surveys['loc_date'] = dutch_surveys.loc_date.map(lambda x: F"{x}")

# make an admin df:
dutch_admin = dutch_surveys[['name','loc_date', *admin]]
a_file_name = F"{project_directory}/dutch_admin_h.csv"
dutch_admin.to_csv(a_file_name)

print("This is the admin data plus a name tag and a loc_date tag\n")
dutch_admin.iloc[:5]

This is the admin data plus a name tag and a loc_date tag



,name,loc_date,Gebiedscode,Rivier,Coordinaten,Lat,Lon,meting,date,doy,dop
0,1,"(1, datetime.date(2017, 10, 21))",W(078a)R-ZHNL,Waal,#REF!,51.8227,5.0159,0,2017-10-21,294.0,293.0
1,2,"(2, datetime.date(2017, 11, 3))",W(074b)R-REFE,Waal,#REF!,51.8290,5.0805,0,2017-11-03,307.0,306.0
2,3,"(3, datetime.date(2017, 10, 16))",W(062a)L-GENL,Waal,#REF!,51.8126,5.2358,0,2017-10-16,289.0,288.0
3,4,"(4, datetime.date(2017, 10, 4))",W(048a)L-GENL,Waal,#REF!,51.8475,5.4159,0,2017-10-04,277.0,276.0
4,5,"(5, datetime.date(2017, 10, 2))",W(046a)L-GENL,Waal,#REF!,51.8588,5.4222,0,2017-10-02,275.0,274.0


In [11]:
print("This is one record:\n")
dutch_admin.iloc[0]

This is one record:



name                                          1
loc_date       (1, datetime.date(2017, 10, 21))
Gebiedscode                       W(078a)R-ZHNL
Rivier                                     Waal
Coordinaten                               #REF!
Lat                                     51.8227
Lon                                      5.0159
meting                                        0
date                                 2017-10-21
doy                                         294
dop                                         293
Name: 0, dtype: object

In [ ]:
# a place to store the results
new_results=[]

def seperate_results(x, a_list):
    # alist is the list of columns to keep
    # return x because we don't want to chage the df
    for element in a_list:
        stuff = {'name':x['name'], 'date':x['date'], 'loc_date':x['loc_date'], 'river':x['Rivier'], 'object':element, 'quantity':x[element] }
        new_results.append(stuff)
    return x

# apply

dutch_surveys.apply(lambda row: seperate_results(row, obj_counts), axis=1)

# make new df 
a_results_df = pd.DataFrame(new_results)
a_file_name = F"{project_directory}/dutch_surveys_h.csv"
a_results_df.to_csv(a_file_name)

print()
print("This is the object results with a  name tag and a loc_date tag\n")
a_results_df.iloc[:10]

In [ ]:
print("The total by loc_date\n")
print(a_results_df.groupby('loc_date').quantity.sum())

In [ ]:
print("The total by river\n")
print(a_results_df.groupby('river').quantity.sum())

In [ ]:
print("The total quantity\n")
print(a_results_df.quantity.sum())

In [ ]:
tops = a_results_df.groupby('object').quantity.sum()
tops = tops.sort_values(ascending=False)

print("The top ten objects:\n")
print(tops[:10])

In [ ]:
# the column names need to be harmonized:
print(a_results_df.columns)
print(swiss_surveys.columns)

In [ ]:
# renaming the columns
a_results_df.rename(columns={'name':'location', 'river':'water_name', 'object':'code'}, inplace=True)
a_results_df['pcs_m'] = a_results_df.quantity/100
a_results_df['pcs_m'] = a_results_df.pcs_m.round(3)


In [ ]:
swiss_surveys.water_name.unique()

In [ ]:
a_min_date = a_results_df['date'].min()
a_max_date = a_results_df['date'].max()
print(a_min_date, a_max_date)

# rivers in switzerland
water_names = ['Aare', 'Sihl', 'Reuss', 'Rhein', 'Limmat', 'Seez']

swiss_surveys['date'] = pd.to_datetime(swiss_surveys['date']).dt.date
swissDf = swiss_surveys[(swiss_surveys.date >= a_min_date)&(swiss_surveys.date <= a_max_date)].copy()

swissDfx = swissDf[swissDf.water_name.isin(water_names)].copy()
survey_total_ch = swissDfx.groupby(['loc_date', 'location', 'date', 'water_name'], as_index=False).agg({'pcs_m':'sum', 'quantity':'sum'})
print("summary stats for the swiss survey totals:\n")
survey_total_ch.describe()

In [ ]:
a_results_dfx = a_results_df.groupby(['loc_date', 'location', 'date', 'water_name'], as_index=False).agg({'pcs_m':'sum', 'quantity':'sum'}).copy()

print("summary stats for the dutch survye totals:\n")
a_results_dfx.describe()

In [ ]:
sns.set_style('whitegrid')

daily_totals = pd.concat([a_results_dfx, survey_total_ch])

fig, ax = plt.subplots(figsize=(8,6))

sns.scatterplot(data=daily_totals, x='date', y='pcs_m', hue='water_name', palette='husl', ax=ax)

ax.set_title(F"Rhine river basin, all daily totals {a_min_date}/{a_max_date}", **title_k)
ax.set_ylabel("Pieces of trash per meter", **ylab_k)
ax.set_xlabel("")

plt.tight_layout()
a_name = F"{project_directory}/initial_harmonized_results.jpg"
plt.savefig(a_name, dpi=300)

plt.show()
plt.close()


### These are the unaccounted for dutch survyes (no date):

In [ ]:
no_date_dutch_surveys

### Change the object definitions to OSPAR codes

The objects definitions need to changed to OSPAR

In [ ]:
## !!! refresh the data from the hammerdirt api here:

# a = requests.get('https://mwshovel.pythonanywhere.com/api/surveys/daily-totals/code-totals/swiss/')
# b = requests.get('https://mwshovel.pythonanywhere.com/api/list-of-beaches/swiss/')
# c = requests.get('https://mwshovel.pythonanywhere.com/api/mlw-codes/list/')

# # the surveys need to be unpacked:
# swiss_surveys = ut.unpack_survey_results(a.json())
# swiss_surveys = pd.DataFrame(swiss_surveys)

# # adding location date column
# swiss_surveys['loc_date'] = list(zip(swiss_surveys['location'], swiss_surveys['date']))

# # hold the original
# x = a.json()

# print("survey columns")
# print(swiss_surveys.columns)

# swiss_beaches = pd.DataFrame(b.json())
# print("beach columns")
# print(swiss_beaches.columns)

# print("code columns")
# swiss_codes = pd.DataFrame(c.json())
# print(swiss_codes.columns)

# swiss_surveys.to_csv(data+'/hammerdirt_data.csv')
# swiss_beaches.to_csv(beaches+'/hammerdirt_beaches.csv')
# swiss_codes.to_csv(codes+'/swiss_codes.csv')
